In [ ]:
class attention(tf.keras.layers.Layer):
    """A class used to build the feed-forward attention layer.

    Attributes
    ----------
    return_sequences: bool, optional
        If False, returns the calculated attention weighted sum of an ECG signal. (default: False)
    dim: int, optional
        The dimension of the attention layer. (default: 64)

    Methods
    -------
    build(input_shape)
        Sets the weights for calculating the attention layer.
    call(x)
        Calculates the attention weights.
    get_config()
        Useful for serialization of the attention layer.

    """

    def __init__(self, return_sequences: bool = False, dim: int = 64, **kwargs) -> None:

        self.return_sequences = return_sequences
        self.dim = dim
        super(attention, self).__init__(**kwargs)

    def build(self, input_shape: Tuple[int, int, int]) -> None:
        """Builds the attention layer.

        alpha = softmax(V.T * tanh(W.T * x + b))

        Parameters
        ----------
        W: tf.Tensor
            The weights of the attention layer.
        b: tf.Tensor
            The bias of the attention layer.
        V: tf.Tensor
            The secondary weights of the attention layer.

        """

        self.W = self.add_weight(
            name="att_weight", shape=(input_shape[-1], self.dim), initializer="normal"
        )
        self.b = self.add_weight(
            name="att_bias", shape=(input_shape[1], self.dim), initializer="zeros"
        )
        self.V = self.add_weight(name="Vatt", shape=(self.dim, 1), initializer="normal")
        super(attention, self).build(input_shape)

    def call(self, x: tf.Tensor) -> tf.Tensor:
        """Calculates the attention weights.

        Parameters
        ----------
        x: tf.Tensor
            The input tensor.

        Returns
        -------
        tf.Tensor
            The attention weighted sum of the input tensor.
        """

        e = K.tanh(K.dot(x, self.W) + self.b)
        e = K.dot(e, self.V)
        a = K.softmax(e, axis=1)
        output = x * a

        if self.return_sequences:
            return output, a

        return K.sum(output, axis=1), a

    def get_config(self) -> Dict[List[Any]]:
        """Returns the config of the attention layer. Useful for serialization."""

        base_config = super().get_config()
        config = {
            "return sequences": tf.keras.initializers.serialize(self.return_sequences),
            "att dim": tf.keras.initializers.serialize(self.dim),
        }
        return dict(list(base_config.items()) + list(config.items()))


In [7]:
from configparser import ConfigParser
from importlib import resources


cfg = ConfigParser()
with resources.path("mulEEG", "config.cfg") as path:
    cfg.read(str(path))

ModuleNotFoundError: No module named 'mulEEG'